## Imports

In [89]:
import pyxdf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lmfit.models import Model
from os import listdir, getcwd
from os.path import isfile, join
from scipy import stats

## Function Definitions


Predicted pupil dilation, $d(Y)$, caused by luminance $Y$, is computed with the following equation: $𝑑(𝑌) = 𝑎 · 𝑒^{−𝑏·𝑌} + c$

In [23]:
def pupil_func(x, a, b, c):
    return a * np.exp(-b * x) + c

In [24]:
def import_data(file):
    streams, header = pyxdf.load_xdf(file)
    dfs = {}
    for stream in streams:
        stream_name = stream['info']['name'][0]
        stream_channels = {channel['label'][0]: i for i, channel in enumerate(stream['info']['desc'][0]['channels'][0]['channel'])}
        stream_data = stream['time_series']
        data_dict = {key: np.array(stream_data)[:, index] for key, index in stream_channels.items()}
        data_dict['time'] = np.around(np.array(stream['time_stamps']), decimals=4)
        dfs[stream_name] = pd.DataFrame(data_dict).drop_duplicates(subset=['time']).reset_index(drop=True)
    return dfs

In [25]:
accom_time = pd.to_timedelta(0.5, unit='s')

In [26]:
def process_gaze_luminance_data(stream_df):
    pupil = stream_df['GazeStream'].loc[(stream_df['GazeStream']['LeftEyeIsBlinking'] == 0) & (stream_df['GazeStream']['RightEyeIsBlinking'] == 0) & (stream_df['GazeStream']['LeftPupilDiameter'] > 0) & (stream_df['GazeStream']['RightPupilDiameter'] > 0), ['time', 'MethodID', 'ModelID', 'LeftPupilDiameter', 'RightPupilDiameter']]
    pupil['time'] = pd.to_timedelta(pupil['time'], unit='s')

    lum = stream_df['LuminanceStream'].loc[:, ['time', 'MethodID', 'ModelID', 'Luminance']]
    lum['time'] = pd.to_timedelta(lum['time'], unit='s')

    # Intersection of time stamps
    pupil_lum_time_intersection = np.intersect1d(pupil['time'], lum['time'])

    # Filter pupil and luminance data by intersection
    pupil = pupil[pupil['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)
    lum = lum[lum['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)

    # Combined DataFrame for pupil and luminance
    pupil_lum = pd.DataFrame({
        'time': pd.to_timedelta(lum['time'], unit='s'),
        'luminance': lum['Luminance'],
        'pupilDiameter': 0.5 * (pupil['LeftPupilDiameter'] + pupil['RightPupilDiameter']),
        'methodID': pupil['MethodID'],
        'modelID': pupil['ModelID']
    }).resample('0.1s', on='time').mean()

    pupil_lum['time'] = pupil_lum.index

    return pupil_lum

In [27]:
def process_calibration_data(pupil_lum_df, stream_df):
    calibration_events = stream_df['ExperimentStream'].loc[(stream_df['ExperimentStream']['EventType'] == 'CalibrationColorChange') | (stream_df['ExperimentStream']['SceneEvent'] == 'Calibration') | (stream_df['ExperimentStream']['SceneEvent'] == 'CalibrationComplete'), ['time','SceneEvent', 'EventType']]
    calibration_events['time'] = pd.to_timedelta(calibration_events['time'], unit='s')
    c_start_times = calibration_events[:8]['time']
    c_end_times = calibration_events[1:]['time']
    c_start_times.reset_index(drop=True, inplace=True)
    c_end_times.reset_index(drop=True, inplace=True)

    calib_data = {}
    for i in range(8):
        calib_data[i] = pupil_lum_df.loc[(pupil_lum_df['time'] >= c_start_times[i]) & (pupil_lum_df['time'] <= c_end_times[i]), ['time','luminance', 'pupilDiameter']]
        calib_data[i]['time'] -= calib_data[i]['time'].iloc[0]
        calib_data[i] = calib_data[i].loc[(calib_data[i]['time'] >= accom_time), ['luminance', 'pupilDiameter']]

    calibration_data = pd.concat(calib_data).groupby(level=0).mean().sort_values(by=['luminance']).reset_index(drop=True)
    return calibration_data

In [28]:
def process_navigation_data(pupil_lum_df, stream_df, a, b, c):
    navigation_events = stream_df['ExperimentStream'].loc[(stream_df['ExperimentStream']['SceneEvent'] == 'NavigationComplete') | (stream_df['ExperimentStream']['SceneEvent'] == 'Navigation_Trial'), ['time','SceneEvent', 'EventType', 'ModelID', 'MethodID']]
    navigation_events['time'] = pd.to_timedelta(navigation_events['time'], unit='s')
    nav_start_times = navigation_events.loc[navigation_events['SceneEvent'] == 'Navigation_Trial', 'time']
    nav_end_times = navigation_events.loc[navigation_events['SceneEvent'] == 'NavigationComplete', 'time']
    nav_start_times.reset_index(drop=True, inplace=True)
    nav_end_times.reset_index(drop=True, inplace=True)

    nav_data = {}
    for i in range(8):
        nav_data[i] = pupil_lum_df.loc[(pupil_lum_df['time']>=nav_start_times.loc[i]) & (pupil_lum_df['time']<=nav_end_times.loc[i]), ['time', 'methodID', 'modelID', 'luminance', 'pupilDiameter']]
        nav_data[i].set_index('time', inplace=True, drop=False)

    navigation_data = pd.concat(nav_data, names=['trial'])
    navigation_data = navigation_data.groupby(level=0).resample('0.5s', on='time', ).mean()
    navigation_data['plr'] = pupil_func(navigation_data['luminance'], a, b, c)
    navigation_data['tepr'] = navigation_data['pupilDiameter'] - navigation_data['plr']
    
    return navigation_data

## Import Data

In [29]:
data_dir = join(getcwd(),'Path_Data')
data_files = [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f))]

In [30]:
dfs = []
for file in data_files:
    dfs.append(import_data(file))

In [80]:
user_ids = []
user_models = []
user_methods= []
user_params = []

for df in dfs:
    pupil_lum_df = process_gaze_luminance_data(df)
    calibration_data = process_calibration_data(pupil_lum_df, df)

    # Fit pupil response to luminance
    x_data = calibration_data['luminance']
    y_data = calibration_data['pupilDiameter']

    exp_mod = Model(pupil_func)
    params = exp_mod.make_params(a=1, b=4, c=0)

    result = exp_mod.fit(y_data, params, x=x_data)
    a = result.params['a'].value
    b = result.params['b'].value
    c = result.params['c'].value

    navigation_data = process_navigation_data(pupil_lum_df, df, a, b, c)

    navigation_avg = navigation_data.groupby(level=0).mean()
    navigation_avg.drop(columns=['luminance'], inplace=True)

    models = navigation_avg.reset_index(drop=True)
    model_avg = models.groupby(['modelID']).mean()
    model_avg.drop(columns=['methodID'], inplace=True)
    #model_avg.reset_index(drop=True, inplace=True)

    methods = navigation_avg.reset_index(drop=True)
    method_avg = methods.groupby(['methodID']).mean()
    method_avg.drop(columns=['modelID'], inplace=True)
    method_avg.reset_index(drop=True, inplace=True)

    user_ids.append(df['ExperimentStream']['UserID'][0]) 
    user_models.append(model_avg)
    user_methods.append(method_avg)
    user_params.append(pd.DataFrame({'params': [a, b, c]}, index=['a', 'b', 'c']))

model_data = pd.concat(user_models, keys=user_ids, names=['UserID'])
method_data = pd.concat(user_models, keys=user_ids, names=['UserID'])
params = pd.concat(user_params, keys=user_ids, names=['UserID'])


In [90]:
# Data Format:
#                   pupilDiameter       plr      tepr
# UserID modelID                                   
# 108    0.0           4.762552  3.896599  0.865953
#        1.0           4.744976  3.944917  0.800058
#        2.0           4.770221  3.877638  0.892583
#        3.0           4.598748  3.838876  0.759872
# 109    0.0           3.313715  2.850761  0.462954
# ...                       ...       ...       ...
# 125    3.0           4.193013  3.863767  0.329246

#check if the data is normally distributed
for i in range(0, len(user_ids)):
    print(stats.shapiro(user_models[i]['tepr']))

ShapiroResult(statistic=0.9469512951187056, pvalue=0.697126785314629)
ShapiroResult(statistic=0.8651583325161332, pvalue=0.27911096712960093)
ShapiroResult(statistic=0.8454164281574257, pvalue=0.21166077730264282)
ShapiroResult(statistic=0.9742544730875243, pvalue=0.8676241782767316)
ShapiroResult(statistic=0.9118800719488307, pvalue=0.4090455077823174)
ShapiroResult(statistic=0.957164990562152, pvalue=0.7940758341792251)
ShapiroResult(statistic=0.6443925018814236, pvalue=0.0008886379421625137)
ShapiroResult(statistic=0.8931716642684863, pvalue=0.25040135429097043)
ShapiroResult(statistic=0.978708556482198, pvalue=0.8944035383301823)
ShapiroResult(statistic=0.8047906988783108, pvalue=0.1110926393784295)
ShapiroResult(statistic=0.9294585401560431, pvalue=0.5912205275352824)
ShapiroResult(statistic=0.8859792701792679, pvalue=0.3648024835631495)
ShapiroResult(statistic=0.9336897023828957, pvalue=0.5502474516522389)
ShapiroResult(statistic=0.889617274759538, pvalue=0.38133764441052276)
Sha